## Learning to Classify Text: Supervised Classification

For this lesson, we will focus on how to build our first automatic classification algorithms. Since the topic is huge, we will be simply scratching the surface, to get something working. For those interested in learning more, taking the Data Mining course next semester is the natural sequence.

Classification is the task of choosing the correct class label for a given input. In basic classification tasks, each input is considered in isolation from all other inputs, and the set of labels is defined in advance. Some examples of classification tasks are:

* Deciding whether an email is spam or not.
* Deciding what the topic of a news article is, from a fixed list of topic areas such as "sports," "technology," and "politics."
* Deciding whether a given occurrence of the word bank is used to refer to a river bank, a financial institution, the act of tilting to the side, or the act of depositing something in a financial institution.

A classifier is called supervised if it is built based on **training data** containing the correct label for each input. 

<img src="http://www.nltk.org/images/supervised-classification.png" width="50%">

(a) During training, we have a set of input cases, for which we know their correct label. Then we take each input and we extract a set of _features_, which capture the basic information about each input. Pairs of feature sets and labels are fed into the machine learning algorithm to generate a model. 

(b) During prediction, we need to classify input for which we do not have the correct label. For that, we extract the  same set of features from the input. we feed these features into the model, which generates predicted labels.


### Gender Identification

Earlier, we have seen how we can generate frequency distribution (`FreqDist`) objects from texts (or collection of texts), and we discussed how such information can be used for identification of important words in a text.

Let's see how we can use these frequency distributions for our first task: Identify the gender of a name.

One more wordlist corpus is the Names corpus, containing 8,000 first names categorized by gender. The male and female names are stored in separate files. Let's find names which appear in both files, i.e. names that are ambiguous for gender:

In [ ]:
import nltk

names = nltk.corpus.names
names.fileids()

In [ ]:
male_names = names.words('male.txt')
print(len(male_names))
print(male_names)

In [ ]:
female_names = names.words('female.txt')
print(len(female_names))
print(female_names)

Now, we need to create our training data. For that, we will create a set of tuples, with the *label* for the name and the actual name:

In [ ]:
data = []
data += [("female", name) for name in female_names] 
data += [("male", name) for name in male_names]

In [ ]:
data[:5]

In [ ]:
data[-5:]

Now, we can build our first rudimentary classifier: We lookup a name in the list, and return the gender in the label.

In [ ]:
def classify_name(input_name):
    for (label, name) in data:
        if name == input_name:
            print(label);

Let's try now our classifier for a few different inputs:

In [ ]:
input_name = "John"
print("Trying ", input_name)
classify_name(input_name)

In [ ]:
input_name = "Jane"
print("Trying ", input_name)
classify_name(input_name)

In [ ]:
input_name = "Leslie"
print("Trying ", input_name)
classify_name(input_name)

In [ ]:
input_name = "Panos"
print("Trying ", input_name)
classify_name(input_name)

Apparently, our classifier has a few problems. Cannot handle at all names that are not in the training data, and has problems when the names appear in both lists. Let's see how many such names there are:

In [ ]:
m = set(male_names)
f= set(female_names)
ambiguous = m & f
print(len(ambiguous))
print(sorted(ambiguous))

One way to improve our classifier is to use a bigger dataset, or count the actual frequency of each name in female and male versions, instead of having just a list. However, none of these solve the underlying problem that the classifier cannot extend beyond the training data.

### Featurization

Featurization is a process in which we represent an input using a set of values, that are derived from the input. 

For example, for gender identification, the last character of the name can give hints about the gender. For example, it is well known that names ending in the letter `a` are almost always female. 

Let's create a revised data set

In [ ]:
last_char_data = [(label, name[-1]) for (label, name) in data]
last_char_data[:10]

In [ ]:
last_char_data[-10:]

Now, we can use the concept of **conditional** frequency distribution, to compare the frequencies of each feature in the two classes:

In [ ]:
cfd = nltk.ConditionalFreqDist(last_char_data)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
# Make the graphs a bit prettier, and bigger
plt.rcParams['figure.figsize'] = (15, 5)

cfd.plot()

This plot shows the number of female and male names ending with each letter of the alphabet; most names ending with a, e or i are female; names ending in h and l are ambiguous and can both male and female; names ending in k, o, r, s, and t are more likely to be male.

Let's build a classifier to model these differences more precisely.

The first step in creating a classifier is deciding what features of the input are relevant, and how to encode those features. For this example, we'll start by just looking at the final letter of a given name. The following feature extractor function builds a dictionary containing relevant information about a given name:

In [ ]:
def gender_features(word):
     return {
        'last_letter': word[-1]
    }

gender_features('Shrek')

Of course, we can add more features if we want. (But beware, as this is not always better, as we will see later.)

In [ ]:
def gender_features(word):
     return {
        'last_letter': word[-1],
        'first_letter': word[0],
        'penultimate_letter': word[-2],
        'last_two_letters': word[-2:]
    }

gender_features('Shrek')

The returned dictionary, known as a feature set, maps from features' names to their values. Feature names are case-sensitive strings that typically provide a short human-readable description of the feature. Feature values are values with simple types, such as booleans, numbers, and strings.

Now that we've defined a feature extractor, we need to prepare a list of examples and corresponding class labels.

In [ ]:
from nltk.corpus import names

def gender_features(word):
     return {
        'last_letter': word[-1]
    }

male_names = names.words('male.txt')
female_names = names.words('female.txt')

labeled_names = []
labeled_names += [("female", name) for name in female_names] 
labeled_names += [("male", name) for name in male_names]

labeled_featuresets = [(gender_features(name), gender) for (gender, name) in labeled_names]

In [ ]:
labeled_featuresets[-5:]

In [ ]:
# We want to randomize the order, before separating into training and test set
import random
random.shuffle(labeled_featuresets)

In [ ]:
labeled_featuresets

In [ ]:
len(labeled_featuresets)

Next, we divide the resulting list of feature sets into a **training set** and a **test set**. The training set is used to train our classifier. The test set will **not** be used for training but only for evaluating the performance of our classifier for "unseen" data that have not been present in the training data.

In [ ]:
# We will keep 500 examples for testing and the remaining ones will be training
train_set, test_set = labeled_featuresets[1000:], labeled_featuresets[:1000]

Now that we have our data ready, let's build our classifier. We will use a "Naive Bayes" classifier. We are not going to talk about the underlying mathematical details of the classification model, and instead will treat as a black box. Covering how the NB classifier works, its strengths and weaknesses, and learning about alternative classification models (e.g., decision trees, logistic regression, support vector machines, etc) is the topic of the Data Mining class.

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

Let's just test it out on some names that did not appear in its training data:

In [ ]:
classifier.classify(gender_features('Neo'))

In [ ]:
classifier.classify(gender_features('Trinity'))

Observe that these character names from The Matrix are correctly classified. Although this science fiction movie is set in 2199, it still conforms with our expectations about names and genders. 

Let's check a few more:

In [ ]:
smalltest = {"Aayush", "Alice", "Anna", "Caroline", "Chanelle", "Christian", "Christine", "Christopher", 
"Colin",  "Dan", "Deanna", "Fred", "Gabe", "Jack", "Jamal", "Jason", "Jeremy", "Jingyi", "Joon", "Joy", 
"Justin", "Karl", "Kelly", "Lori", "Luke", "Malachi", "Maureen", "Max", "Orion", "Rosanna", "Sachin", 
"Sarah", "Shane", "Sherman", "Shirley", "Sihao", "Sooho", "Stephanie", "Steven", "Tomas", "Tomer", "Tracy", 
"Vinit", "Xinhui", "Yangsin", "Zachary", "Panos", "Karthik"
 }

for name in smalltest:
    features = gender_features(name)
    print("Name: ", name, " ==> ", classifier.classify(features))
    

We can systematically evaluate the classifier on a much larger quantity of unseen data:

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

Finally, we can examine the classifier to determine which features it found most effective for distinguishing the names' genders:

In [ ]:
classifier.show_most_informative_features(26)

#### Exercise 

Modify the gender_features() function to provide the classifier with features encoding the length of the name, its first letter, and any other features that seem like they might be informative. Retrain the classifier with these new features, and test its accuracy.

In [ ]:
# your code here
from nltk.corpus import names
import random

# Modify this function to add more features
def gender_features(word):
    return {
        'last_letter': word[-1],        
    }

male_names = names.words('male.txt')
female_names = names.words('female.txt')

labeled_names = []
labeled_names += [("female", name) for name in female_names] 
labeled_names += [("male", name) for name in male_names]

labeled_featuresets = [(gender_features(name), gender) for (gender, name) in labeled_names]

# We are going to repeat the process multiple times, as the shuffling generates different 
# sets of training and test data
train_set, test_set = [], []
trials = 50
psum = 0;
cnt = 0;
for i in range(trials):
    random.shuffle(labeled_featuresets)
    # We will keep 500 examples for testing and the remaining ones will be training
    train_set, test_set = labeled_featuresets[500:], labeled_featuresets[:500]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    accuracy = nltk.classify.accuracy(classifier, test_set)
    # print("Trial:", cnt, " Accuracy:", accuracy)
    psum += accuracy
    cnt += 1
    
print("Avg Accuracy: ", (psum/cnt))

In [ ]:
classifier.show_most_informative_features(100)

**"Big" data note**: When working with large data sets, constructing a single list that contains the features of _every_ instance can use up a large amount of memory. In these cases, use the function `nltk.classify.apply_features`, which returns an object that acts like a list but does not store all the feature sets in memory:

In [ ]:
from nltk.classify import apply_features
random.shuffle(labeled_names)
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

### Choosing the right features

Selecting relevant features and deciding how to encode them for a learning method can have an enormous impact on the learning method's ability to extract a good model. Much of the interesting work in building a classifier is deciding what features might be relevant, and how we can represent them. Although it's often possible to get decent performance by using a fairly simple and obvious set of features, there are usually significant gains to be had by using carefully constructed features based on a thorough understanding of the task at hand.

Typically, feature extractors are built through a process of trial-and-error, guided by intuitions about what information is relevant to the problem. It's common to start with a "kitchen sink" approach, including all the features that you can think of, and then checking to see which features actually are helpful.

In [ ]:
def gender_features_expanded(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

print(gender_features_expanded('Panos'))

However, there are usually limits to the number of features that you should use with a given learning algorithm — if we provide too many features, then the algorithm will have a higher chance of relying on idiosyncrasies of your training data that don't generalize well to new examples. This problem is known as **overfitting**, and can be especially problematic when working with small training sets. 

For example, if we train a naive Bayes classifier using the feature extractor shown in 1.2, it will overfit relatively small training set, resulting in a system whose accuracy is lower than the accuracy of a classifier that only pays attention to the final letter of each name:

In [ ]:
random.shuffle(labeled_names)
featuresets = [(gender_features_expanded(n), gender) for (gender, n) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
train_names, test_names = labeled_names[500:], labeled_names[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(20)

So, let's keep our original classifier

In [ ]:
random.shuffle(labeled_names)
featuresets = [(gender_features(n), gender) for (gender, n) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
train_names, test_names = labeled_names[500:], labeled_names[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

We can generate a list of the errors that the classifier makes when predicting name genders:

In [ ]:
errors = []
for (correct, name) in test_names:
    guess = classifier.classify(gender_features(name))
    if correct != guess:
        errors.append( (correct, guess, name) )

In [ ]:
len(errors)

In [ ]:
errors

#### Confusion Matrix

If we want to learn more about the specific types of errors for our classifier, we can create a "confusion matrix". A confusion matrix shows the number of times that a classifier classifies a specific instance into a particular class (E.g., males as males, males as females, etc).

In [ ]:
gold = [gender for (features,gender) in test_set]

In [ ]:
guess = [classifier.classify(features) for (features, gender) in test_set]

In [ ]:
gold = [gender for (features,gender) in test_set]
guess = [classifier.classify(features) for (features, gender) in test_set]

cm = nltk.ConfusionMatrix(gold, guess)

In [ ]:
print(cm.pretty_format(sort_by_count=True, show_percents=False))

In [ ]:
print(cm.pretty_format(sort_by_count=True, show_percents=True))

We can then examine individual error cases where the model predicted the wrong label, and try to determine what additional pieces of information would allow it to make the right decision (or which existing pieces of information are tricking it into making the wrong decision). The feature set can then be adjusted accordingly. The names classifier that we have built generates about 100 errors on the test corpus:

In [ ]:
for (correct, guess, name) in sorted(errors):
    print('correct=%-8s guess=%-8s name=%-30s' % (correct, guess, name))

Looking through this list of errors makes it clear that some suffixes that are more than one letter can be indicative of name genders. For example, names ending in yn appear to be predominantly female, despite the fact that names ending in n tend to be male; and names ending in ch are usually male, even though names that end in h tend to be female. We therefore adjust our feature extractor to include features for two-letter suffixes:

In [ ]:
def gender_features(word):
    return {'last_letter': word[-1:],
           'last_two_letters': word[-2:]}

random.shuffle(labeled_names)
featuresets = [(gender_features(n), gender) for (gender, n) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
train_names, test_names = labeled_names[500:], labeled_names[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

This error analysis procedure can then be repeated, checking for patterns in the errors that are made by the newly improved classifier. Each time the error analysis procedure is repeated, we should select a different test/training split, to ensure that the classifier does not start to reflect idiosyncrasies in the test set.

### Document Classification

A common classification task is to classify documents into categories. Let's use for this the Movie Reviews corpus from NLTK:

In [ ]:
from nltk.corpus import movie_reviews

categories = movie_reviews.categories()
categories

Now let's generate the list of files, each with its corresponding category.

In [ ]:
labeled_files = []
for category in categories:
    labeled_files += [(fileid, category) for fileid in movie_reviews.fileids(category)]

In [ ]:
labeled_files[:5]

In [ ]:
labeled_files[-5:]

In [ ]:
len(labeled_files)

In [ ]:
len([l for l in labeled_files if l[1]=='pos'])

In [ ]:
len([l for l in labeled_files if l[1]=='neg'])

Now let's create the features. We will create one feature per word, with a binary value, indicating whether the document contains the word or not. To limit the number of features that the classifier needs to process, we begin by constructing a list of the 2000 most frequent words in the overall corpus

In [ ]:
import nltk
from nltk.corpus import stopwords
import random

stop_words = stopwords.words('english')

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words() if w.isalpha() and w not in stop_words)
word_features = [w for (w,f) in all_words.most_common(2000)]

def document_features(fileid):
    document_words = set(movie_reviews.words(fileid))
    features = {}
    for word in word_features:
        features[word] = (word.lower() in document_words)
    return features

In [ ]:
%timeit  document_features("pos/cv995_21821.txt")

In [ ]:
punct = ["!", "."]
tokens = list(movie_reviews.words("pos/cv995_21821.txt"))
data = [(w, tokens.count(w)) for w in punct]
data

In [ ]:
document_features("pos/cv995_21821.txt")

In [ ]:
labeled_documents = [(document_features(fileid), category) for (fileid, category) in labeled_files]
random.shuffle(labeled_documents)
train_set, test_set = labeled_documents[100:], labeled_documents[:100]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(50)

### Exercise

Try to come up with features that will improve the classifier that we discussed above. 

In [ ]:
# Modify the code below

import nltk
from nltk.classify import apply_features
from nltk.corpus import stopwords
import random

stop_words = stopwords.words('english')
# Adding actor names in the list of stopwords
stop_words.append("seagal")
stop_words.append("damon")
stop_words.append("jedi")
stop_words.append("anna")
stop_words.append("hanks")
stop_words.append("lucas")
stop_words.append("ripley")
stop_words.append("mulan")

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words() if w.isalpha()  and w not in stop_words)
word_features = [w for (w,f) in all_words.most_common(2000) ]

def document_features(fileid):
    # Modify to add your own set of features
    document_words = set(movie_reviews.words(fileid))
    features = {}
    for word in word_features:
        features[word] = (word.lower() in document_words)
    return features

random.shuffle(labeled_files)
train_set = apply_features(document_features, labeled_files[1000:])
test_set = apply_features(document_features, labeled_files[:1000])
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Accuracy:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(50)